In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import time
from datetime import datetime
from config import weather_api_key
import requests
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
coordinates = list(lat_lngs)

In [5]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

643

In [6]:
city_data = []

print("Beginning Data Retrieval       ")
print("-------------------------------")

record_count = 1
set_count = 1

for i, city in enumerate(cities):
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count += 1
        time.sleep(60)
    city_url = url + "&q=" + city.replace(" ", "+")

    print(f"Processing Record {record_count} of Set {set_count}  |  {city}")
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval       
-------------------------------
Processing Record 1 of Set 1  |  vaini
Processing Record 2 of Set 1  |  tazovskiy
Processing Record 3 of Set 1  |  dikson
Processing Record 4 of Set 1  |  ponta do sol
Processing Record 5 of Set 1  |  ushuaia
Processing Record 6 of Set 1  |  hithadhoo
Processing Record 7 of Set 1  |  concepcion del oro
Processing Record 8 of Set 1  |  sicamous
Processing Record 9 of Set 1  |  bombay
Processing Record 10 of Set 1  |  windhoek
Processing Record 11 of Set 1  |  cam ranh
Processing Record 12 of Set 1  |  belushya guba
City not found. Skipping...
Processing Record 13 of Set 1  |  thompson
Processing Record 14 of Set 1  |  meulaboh
Processing Record 15 of Set 1  |  tuktoyaktuk
Processing Record 16 of Set 1  |  rikitea
Processing Record 17 of Set 1  |  ambilobe
Processing Record 18 of Set 1  |  busselton
Processing Record 19 of Set 1  |  neustrelitz
Processing Record 20 of Set 1  |  olafsvik
Processing Record 21 of Set 1  |  cock

In [7]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vaini,-21.2000,-175.2000,71.76,94,100,6.91,TO,2022-04-04 17:32:42
1,Tazovskiy,67.4667,78.7000,25.99,95,100,23.76,RU,2022-04-04 17:32:43
2,Dikson,73.5069,80.5464,-4.05,97,14,7.58,RU,2022-04-04 17:32:43
3,Ponta Do Sol,32.6667,-17.1000,62.71,54,43,11.77,PT,2022-04-04 17:32:44
4,Ushuaia,-54.8000,-68.3000,44.22,81,75,28.77,AR,2022-04-04 17:29:29
5,Hithadhoo,-0.6000,73.0833,83.73,67,48,3.58,MV,2022-04-04 17:32:44
6,Concepcion Del Oro,24.6333,-101.4167,77.20,8,8,16.64,MX,2022-04-04 17:32:44
7,Sicamous,50.8331,-118.9856,56.37,97,100,3.20,CA,2022-04-04 17:28:15
8,Bombay,19.0144,72.8479,84.18,79,40,5.75,IN,2022-04-04 17:31:57
9,Windhoek,-22.5594,17.0832,71.80,33,0,8.05,NA,2022-04-04 17:29:13


In [8]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Vaini,TO,2022-04-04 17:32:42,-21.2000,-175.2000,71.76,94,100,6.91
1,Tazovskiy,RU,2022-04-04 17:32:43,67.4667,78.7000,25.99,95,100,23.76
2,Dikson,RU,2022-04-04 17:32:43,73.5069,80.5464,-4.05,97,14,7.58
3,Ponta Do Sol,PT,2022-04-04 17:32:44,32.6667,-17.1000,62.71,54,43,11.77
4,Ushuaia,AR,2022-04-04 17:29:29,-54.8000,-68.3000,44.22,81,75,28.77
5,Hithadhoo,MV,2022-04-04 17:32:44,-0.6000,73.0833,83.73,67,48,3.58
6,Concepcion Del Oro,MX,2022-04-04 17:32:44,24.6333,-101.4167,77.20,8,8,16.64
7,Sicamous,CA,2022-04-04 17:28:15,50.8331,-118.9856,56.37,97,100,3.20
8,Bombay,IN,2022-04-04 17:31:57,19.0144,72.8479,84.18,79,40,5.75
9,Windhoek,NA,2022-04-04 17:29:13,-22.5594,17.0832,71.80,33,0,8.05


In [9]:
output_data_file = "weather_data/cities.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")